In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
from keras.utils import to_categorical
from nltk.tag import hmm
from nltk.probability import LaplaceProbDist
import random

from finger_letter_mapping import letter_finger, finger_letter

C:\Users\Home\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import nltk
nltk.download('abc')
nltk.download('punkt')

[nltk_data] Downloading package abc to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package abc is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from smartcheck.smartcheck import Smartcheck

In [4]:
def wordsplit_to_sent(data) : 
    return [' '.join(sent)[:-2].lower() for sent in data]

In [5]:
sentences_wordsplit = nltk.corpus.abc.sents()[:9000]
tester_wordsplit = nltk.corpus.abc.sents()[10000:11000]

In [6]:
train_sentences = wordsplit_to_sent(sentences_wordsplit)
test_sentences = wordsplit_to_sent(tester_wordsplit)

In [7]:
train_sentences[:2]

['pm denies knowledge of awb kickbacks the prime minister has denied he knew awb was paying kickbacks to iraq despite writing to the wheat exporter asking to be kept fully informed on iraq wheat sales',
 'letters from john howard and deputy prime minister mark vaile to awb have been released by the cole inquiry into the oil for food program']

In [8]:
test_sentences[:2]

['compo for murray valley irrigators ruled out the new south wales government has ruled out compensation for murray valley irrigators affected by water cut backs',
 'high security allocations and carry over water in the murray valley have been cut by 20 per cent because of record low inflows']

In [9]:
training = train_sentences
testing = test_sentences

In [10]:
# f1 = open('training.txt', 'r')
# f2 = open('testing.txt', 'r')
# training = f1.read().lower().splitlines()
# testing = f2.read().lower().splitlines()
# f1.close()
# f2.close()

In [11]:
print('Letter space (hidden states):\n ', letter_finger.keys())
print('Finger space (observed states): \n', finger_letter.keys())

Letter space (hidden states):
  dict_keys(['q', 'a', 'z', 'w', 's', 'x', 'e', 'd', 'c', 'r', 'f', 'v', 't', 'g', 'b', 'p', 'o', 'l', 'i', 'k', 'u', 'j', 'm', 'y', 'h', 'n', ' '])
Finger space (observed states): 
 dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8])


In [12]:
print(training[0])

pm denies knowledge of awb kickbacks the prime minister has denied he knew awb was paying kickbacks to iraq despite writing to the wheat exporter asking to be kept fully informed on iraq wheat sales


In [13]:
def text_to_finger(data): 
    output = [] 
    for row in data:
        output.append(np.array([letter_finger[c] for c in row.lower() if c in letter_finger.keys()]))
    return np.array(output)
def text_to_label(data): 
    output = []
    for row in data: 
        output.append(np.array(list(filter(lambda c: c in letter_finger.keys(), row))))
    return np.array(output)

In [14]:
mapped_training = text_to_finger(training)
mapped_testing = text_to_finger(testing)
labels_training = text_to_label(training)
labels_testing = text_to_label(testing)

# list(map(lambda c: letter_finger[c] if c in letter_finger.keys() else None, sample_text.lower()))
print('reality check:', mapped_training.size, len(labels_training))
print(mapped_training[0].size, labels_training[0].size)

reality check: 9000 9000
198 198


In [15]:
def finger_to_onehot(data): 
    return np.array([to_categorical(row) for row in data])

In [16]:
onehot_training = finger_to_onehot(mapped_training)
onehot_testing = finger_to_onehot(mapped_testing)

In [17]:
# TO DO: add noise to one hot encoding 

# to turn one hot back to label: np.argmax(mapped_one_hot[0])
# mapped_noise = [letter + NOISE for letter in mapped_one_hot]
# for letter in mapped_one_hot: 
    

turn finger mapping back to text using HMM

In [18]:
trainer = hmm.HiddenMarkovModelTrainer(states = letter_finger.keys(), symbols = finger_letter.keys())
# symbols: observations ; states: hidden states

In [19]:
est = LaplaceProbDist

In [20]:
def make_seq(finger, labels): 
    out = []
    for i in range(len(finger)): 
        out.append([(ss, target) for ss, target in zip(finger[i], labels[i])])
    return out

In [21]:
seq = make_seq(mapped_training, labels_training)
test_seq = make_seq(mapped_testing, labels_testing)

In [22]:
tagger = trainer.train_supervised(seq, estimator=est)

In [23]:
tagger.test(test_seq)

accuracy over 133408 tokens: 78.84


In [24]:
def letters_to_sents(tuple_list):  # turn [[(finger, letter), (.,.)], [...]] to list of sentences 
    return [''.join([y for (x,y) in entry]) for entry in tuple_list]

In [25]:
test_true_output = letters_to_sents(test_seq)

In [26]:
test_input = [[x for (x,y) in entry] for entry in test_seq]

In [27]:
test_out = [tagger.tag(sent) for sent in test_input]

In [28]:
test_out_sents = letters_to_sents(test_out)

In [37]:
% cd smartcheck 
% pwd
checker = Smartcheck()

[WinError 2] The system cannot find the file specified: 'smartcheck'
C:\Users\Home\Desktop\Projects\NeuroTech2020\NeuroTech-ML\siggy\smartcheck


In [51]:
checker.correction('biok', 'the')

bigot 4.391368203096302e-07 1e-10
broo 4.9596665124121945e-08 1e-10
bi 2.2223262622723224e-05 1.1551103000345158e-06
bock 1.148594084061379e-06 1e-10
bijou 5.814315892164282e-07 1e-10
bark 6.053040969589862e-06 1.302612101137252e-06
hook 1.931454038126194e-05 4.22335521566545e-06
brink 2.5247710416140792e-06 3.4671607894329143e-06
bide 3.287468758530712e-07 1e-10
boor 2.2994972041280564e-07 1e-10
boil 4.118628202779706e-06 6.670876228463609e-07
biro 3.151307047702789e-07 1e-10
oink 3.6395032316802273e-07 1e-10
blog 0.0002466169918216982 1.1679845989445775e-05
bo 1.063674779115699e-05 1e-10
ilk 7.113038124275619e-07 1e-10
pion 6.89276153039753e-07 1e-10
amok 7.883572620977541e-07 1e-10
wick 2.081789795378101e-06 1e-10
blob 1.7950697552709562e-06 1e-10
pick 6.553403460878577e-05 1.6356911010997643e-06
ion 2.052997748700248e-05 1.5498281924338301e-06
boom 1.3380073001411039e-05 1.950148286122249e-06
dink 4.3403755743784024e-07 1e-10
sook 1.9532438225971096e-07 1e-10
milk 3.563177349393442

'back'

In [47]:
words = letters_to_sents([tagger.tag([letter_finger[c] for c in 'hello this is a testing open computer'])])[0].split()

In [48]:
words

['helll', 'this', 'is', 'a', 'besting', 'open', 'compurer']

In [ ]:
[checker.correction()for ]

In [28]:
# from spell import correction
# letters_to_sents([tagger.tag([letter_finger[c] for c in 'hello this is a testing open computer'])])[0]